# Imports
Всякие нужные импорты и установки: работаю в коллабе, так что каждый раз заново...

In [1]:
!pip3 install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip3 install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.3 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=718ed8e88cde9ad7a24039ee3d757db66605d60cd31c33fe75485d240ae28da2
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [3]:
!pip3 install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 237 kB 45.6 MB/s 
     |████████████████████████████████| 51 kB 7.1 MB/s 
     |████████████████████████████████| 5.5 MB 50.4 MB/s 
     |████████████████████████████████| 1.3 MB 38.4 MB/s 
     |████████████████████████████████| 182 kB 41.6 MB/s 
     |████████████████████████████████| 7.6 MB 37.3 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23799 sha256=a4097406c1045df980d19ab463eb0ece8e2a57c8422de207af6f8d7a49cbd131
  Stored in directory: /root/.cache/pip/wheels/85/0d/12/77d219f3ebbb22dc22234b4d665886c0eace86a26eca0aa72b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7ab7fca23a1b04b76a965b2ec2e103bcb424c16d77eadfa156f1cc7d797712db
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59

In [5]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.6 MB/s 
     |████████████████████████████████| 8.2 MB 6.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=0ef6e6898dfa35008c1e7d3307bac9eabd28ed910df8f05a3e9154275ea41092
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [157]:
import re
import pymorphy2
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from summa import keywords
from keybert import KeyBERT
import pandas as pd
from pprint import pprint

morph = pymorphy2.MorphAnalyzer()
stop = stopwords.words('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Reaing
Читаем данные

In [8]:
path = '/content/drive/MyDrive/nnlp/nlp4/data.txt'

In [9]:
with open(path, encoding='utf-8') as f:
    text = f.read()

# Classes
Классы для удобства работы. По хорошему, там должны быть датаклассы, но там тоже неплохо

In [10]:
class Word:
    def __init__(self, word=None, lemma=None, pos=None, BIO_manual=None, 
                 BIO_tag=None, BIO=None):
        self.word = word
        self.lemma = lemma
        self.pos = pos
        self.BIO_manual = BIO_manual
        self.BIO_tag = BIO_tag
        self.BIO = BIO

In [11]:
class Sentence:
    def __init__(self, raw=None, words=None):
        self.raw = raw
        if not words:
            self.words = []
        else:
            self.words = words

    def add_word(self, word):
        self.words.append(word)

In [12]:
class Text:
    def __init__(self, sents=None, tags=None):
        self.tags = tags
        if not sents:
            self.sentences = []
        else:
            self.sentences = sents

    def add_sent(self, sent):
        self.sentences.append(sent)

# Processing functions
Функции для препроцессинга, построения нужной структуры данных и извлечения ключевых слов из разметки так, чтобы они в нее легли

In [13]:
def process_word(word):
    m = morph.parse(word)[0]
    pos = m.tag.POS
    lemma = m.normal_form
    return lemma, pos

In [14]:
def process_input_file(text):
    pattern = re.compile(r'[\w-]+')
    texts = [el.strip() for el in text.split('***')]
    for i, text in enumerate(texts):
        text = sent_tokenize(text)
        res = Text()
        for sent in text:
            for line in sent.splitlines():
                if line.startswith('#'):
                    line = line.replace('# ', '').split(', ')
                    new_line = []
                    for tag in line:
                        els = ' '.join([process_word(el)[0] for el in tag.split()])
                        new_line.append(els)
                    res.tags = new_line
                else:
                    raw = line.replace('-B', '').replace('-I', '')
                    sent = Sentence(raw=raw)
                    for el in re.findall(pattern, line):
                        word = Word(word=el.lower())
                        sent.add_word(word)                   
                    res.add_sent(sent)
        texts[i] = res
    return texts

In [83]:
def get_manual_kw(data):
    res = []
    for text in data:
        new_text = Text(tags=text.tags)
        for sent in text.sentences:
            new_sent = Sentence(raw=sent.raw)
            for word in sent.words:
                lemma, pos = process_word(word.word.replace('-b', '').replace('-i', ''))
                if '-b' in word.word:
                    bio = 'B'
                elif '-i' in word.word:
                    bio = 'I'
                else:
                    bio = None
                new_word = Word(word=word, lemma=lemma, pos=pos, BIO_manual=bio)
                new_sent.add_word(new_word)
            new_text.add_sent(new_sent)
        res.append(new_text)
    return res

In [84]:
def map_unigrams(sent, tags):
    unis = [el for el in tags if len(el.split()) == 1]
    for i, word in enumerate(sent.words):
        if word.lemma in unis:
            word.BIO_tag = 'B'
        sent.words[i] = word
    return sent

In [85]:
def map_bigrams(sent, tags):
    bis = [tuple(el.split()) for el in tags if len(el.split()) == 2]
    for i, word in enumerate(sent.words[:-1]):
        if (word.lemma, sent.words[i + 1].lemma) in bis:
            word.BIO_tag = 'B'
            sent.words[i + 1].BIO_tag = 'I'
        sent.words[i] = word
    return sent

In [86]:
def get_tags_kw(data):
    res = []
    for text in data:
        new_text = Text(tags=text.tags)
        tags = text.tags
        for sent in text.sentences:
            new_sent = map_unigrams(sent, ['устройство'])
            new_sent = map_bigrams(sent, tags)
            new_text.add_sent(new_sent)
        res.append(new_text)
    return res

In [87]:
def get_total_kw(data):
    for ntext in range(len(data)):
        for nsent in range(len(data[ntext].sentences)):
            for nword in range(len(data[ntext].sentences[nsent].words)):
                word = data[ntext].sentences[nsent].words[nword]
                if word.BIO_manual is not None:
                    data[ntext].sentences[nsent].words[nword].BIO = word.BIO_manual
                elif word.BIO_tag is not None:
                    data[ntext].sentences[nsent].words[nword].BIO = word.BIO_tag
    return data

Запускаем все вышеперечисленное

In [88]:
data = process_input_file(text)

In [89]:
data = get_manual_kw(data)

In [90]:
data = get_tags_kw(data)

In [91]:
data = get_total_kw(data)

In [22]:
def make_text(data):
    texts = []
    for text in data:
        cur = []
        for sent in text.sentences:
            for word in sent.words:
                cur.append(word.lemma)
        texts.append(' '.join(cur))
    return texts

In [23]:
clean_texts = make_text(data)
raw_texts = [el.replace('-B', '').replace('-I', '').strip() for el in text.split('***')]

# RAKE

In [24]:
rake = RAKE.Rake(stop)
rake_lst_raw = []
for el in raw_texts:
    rake_lst_raw.append(rake.run(el, maxWords=3, minFrequency=2))
rake_lst_raw

[[('электронных компонентов', 4.833333333333334),
  ('кустарные производства', 4.0),
  ('это', 1.875),
  ('строя', 1.3333333333333333),
  ('продают', 1.25),
  ('т', 1.0),
  ('п', 1.0),
  ('далеко', 1.0),
  ('китае', 1.0),
  ('россии', 1.0),
  ('например', 1.0),
  ('платы', 1.0),
  ('целом', 1.0),
  ('проблемы', 1.0),
  ('—', 0)],
 [('usb\n\nмаксимальная диагональ', 7.5),
  ('1\n\nпиковая яркость', 4.0),
  ('3500\n\nпроекционная технология', 4.0),
  ('3000\n\nпиковая яркость', 4.0),
  ('usb', 2.0),
  ('60\n\nцена', 1.5),
  ('менее', 1.3333333333333333),
  ('9\n\nинтерфейс', 1.2),
  ('увы', 1.0),
  ('кстати', 1.0),
  ('2010 году', 1.0),
  ('случилось', 1.0),
  ('лм', 1.0),
  ('16', 0)],
 [('санкт-петербурге', 4.0),
  ('наборы', 1.75),
  ('рынке', 1.5),
  ('программировать', 1.0),
  ('javascript', 1.0),
  ('java', 1.0)],
 [('привязаны ваши счета', 8.333333333333334),
  ('подписки', 1.3333333333333333),
  ('карты»', 1.25),
  ('показывает', 1.0)],
 [('платы', 1.6666666666666667),
  ('также'

In [25]:
rake = RAKE.Rake(stop)
rake_lst_cl = []
for el in clean_texts:
    rake_lst_cl.append(rake.run(el, maxWords=3, minFrequency=2))
rake_lst_cl

[[('кустарный производство', 4.583333333333334),
  ('это', 2.2222222222222223),
  ('проблема', 1.6666666666666667),
  ('данные', 1.3333333333333333),
  ('продавать', 1.25),
  ('п', 1.0),
  ('далеко', 1.0),
  ('китай', 1.0),
  ('россия', 1.0),
  ('строй', 1.0)],
 [('пико-проектор', 6.023809523809524), ('2010 год', 1.0), ('случиться', 1.0)],
 [('российский разработка', 4.8),
  ('рынок', 2.0),
  ('основать', 1.0),
  ('2010 год', 1.0),
  ('javascript', 1.0),
  ('java', 1.0)],
 [('привязать ваш счёт', 9.0),
  ('карта', 1.6666666666666667),
  ('подписка', 1.0)],
 [('переработка', 1.3333333333333333), ('из-', 1.3333333333333333)],
 [('онлайн-площадка', 4.333333333333334),
  ('это', 2.3846153846153846),
  ('пользователь', 2.1666666666666665),
  ('бизнес', 1.9),
  ('нагрузка', 1.8333333333333333),
  ('этап', 1.75),
  ('значит', 1.6666666666666667),
  ('каждый', 1.5),
  ('проблема', 1.5),
  ('работа', 1.3333333333333333),
  ('ситуация', 1.3333333333333333),
  ('вопрос', 1.0),
  ('статья', 1.0),


# TextRank

In [26]:
trank_lst_raw = []
for el in raw_texts:
    trank_lst_raw.append(keywords.keywords(el, language='russian', additional_stopwords=stop, scores=True))
trank_lst_raw

[[('которой', 0.2962260155939893),
  ('которая', 0.2962260155939893),
  ('компоненты', 0.26966732203493377),
  ('компонентов', 0.26966732203493377),
  ('компонент', 0.26966732203493377),
  ('которые этим', 0.24526941279192954),
  ('электронном', 0.20991871565913117),
  ('электронный', 0.20991871565913117),
  ('это', 0.19431280998986977),
  ('эта', 0.19431280998986977),
  ('компаний', 0.18446858308238115),
  ('компаниям', 0.18446858308238115),
  ('накопителя который', 0.17973916248742652),
  ('электронные отходы', 0.1715439802021218),
  ('устройств', 0.14948001037982678),
  ('устройства', 0.14948001037982678),
  ('устройство', 0.14948001037982678),
  ('техники', 0.14342980106543654),
  ('выбрасывают', 0.14215359558925553),
  ('выбрасывать', 0.14215359558925553),
  ('объему электронных отходов', 0.1352250781305513),
  ('утилизацию', 0.13511445178125436),
  ('утилизации', 0.13511445178125436),
  ('утилизация', 0.13511445178125436),
  ('тонн электронного мусора', 0.1335501683499056),
  ('с

In [27]:
trank_lst_cl = []
for el in clean_texts:
    trank_lst_cl.append(keywords.keywords(el, language='russian', additional_stopwords=stop, scores=True))
trank_lst_cl


[[('редко публиковаться статья', 0.5423766856565608),
  ('буквально тысяча', 0.03355159815570825),
  ('элементарный поломка', 0.023542347180555995),
  ('научиться контролировать', 0.022657738370059147),
  ('похоже начинать задумываться', 0.02067279534671559),
  ('минус', 0.015105158913372779),
  ('стоить оно немалый деньга поэтому', 0.010784980413222076),
  ('лицензированный предприятие', 0.008519999009692397),
  ('пара день', 0.0066996148756889645),
  ('давно коммерсант опубликовать', 0.005000033299273428),
  ('факт являться', 0.003654238248195253),
  ('новость', 0.00300001997956406),
  ('нерешаемый', 0.0024361588321301655),
  ('компонент', 1.0718974215337562e-15),
  ('это', 7.577822791779828e-16),
  ('вроде смартфон ноутбук телевизор', 3.7824922873462557e-16),
  ('общенациональный', 3.09130012724421e-16),
  ('данные', 3.0781097693813696e-16),
  ('данный момент', 3.011842271935437e-16),
  ('выпаивать', 2.4682601389294163e-16),
  ('запуск программа', 2.4437955564490816e-16),
  ('экспер

# KBERT

In [28]:
kw_model = KeyBERT('clips/mfaq')

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:370: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [29]:
kbert_lst_raw = kw_model.extract_keywords(raw_texts, keyphrase_ngram_range=(1, 2))
kbert_lst_raw

[[('отходов понятно', 0.9703),
  ('электронных отходов', 0.9694),
  ('таких отходов', 0.9691),
  ('отходов год', 0.969),
  ('отходов', 0.9687)],
 [('проекторы iphone', 0.9668),
  ('создателям пришлось', 0.9655),
  ('году пикопроекторы', 0.9644),
  ('iphone', 0.9644),
  ('подобные гаджеты', 0.9641)],
 [('российских разработок', 0.9696),
  ('robotics', 0.969),
  ('международные марки', 0.9685),
  ('робототехнические наборы', 0.9685),
  ('эти робототехнические', 0.9683)],
 [('ваши счета', 0.9635),
  ('все счета', 0.9618),
  ('карты куда', 0.9618),
  ('кредит разделе', 0.9617),
  ('отслеживания привязанных', 0.9615)],
 [('ситуация утилизации', 0.9661),
  ('электроника утилизация', 0.9654),
  ('утилизации', 0.9652),
  ('половина устройств', 0.9649),
  ('утилизация', 0.9644)],
 [('общаюсь заказчиками', 0.9753),
  ('проектировать систему', 0.9751),
  ('бизнес модернизирует', 0.9749),
  ('бизнес процессы', 0.9748),
  ('развития бизнес', 0.9746)]]

In [30]:
kbert_lst_cl = kw_model.extract_keywords(clean_texts, keyphrase_ngram_range=(1, 2))
kbert_lst_cl

[[('цивилизация каждый', 0.9695),
  ('известно что', 0.9694),
  ('так многие', 0.9692),
  ('мусор перспектива', 0.9691),
  ('наш цивилизация', 0.969)],
 [('пикопроектор новинка', 0.9727),
  ('любопытный вариант', 0.972),
  ('пико проектор', 0.9718),
  ('проектор iphone', 0.9717),
  ('пикопроектор при', 0.9716)],
 [('робототехника собственный', 0.9737),
  ('робототехнический тут', 0.9735),
  ('именно робототехнический', 0.9734),
  ('программировать робот', 0.9729),
  ('robot конструктор', 0.9724)],
 [('новый функция', 0.9731),
  ('функция', 0.9703),
  ('приложение сбербанк', 0.9703),
  ('райффайзенбанк функция', 0.97),
  ('добавить возможность', 0.9698)],
 [('ситуация утилизация', 0.9702),
  ('увеличиться количество', 0.9671),
  ('ситуация', 0.967),
  ('техника теперь', 0.9669),
  ('электроника утилизация', 0.9667)],
 [('разрабатывать свой', 0.9739),
  ('создавать интернет', 0.9739),
  ('разработать', 0.9737),
  ('бизнес процесс', 0.9734),
  ('задача бизнес', 0.9733)]]

# Patterns
Получаем по шаблонам возможных кандидатов. Все в кучу, потому что потом все равно мы не сможем в ключевых словах найти того, чего там нет.

In [36]:
patterns = [('ADJS', 'NOUN'), ('NOUN', 'NOUN'), ('NOUN', ), ('COMP', 'NOUN'),
            ('ADJF', 'NOUN')]

In [37]:
def get_mapped_patterns(data, patterns):
    res = []
    for text in data:
        for sent in text.sentences:
            for i, word in enumerate(sent.words[:-1]):
                if (word.pos, sent.words[i + 1].pos) in patterns:
                    res.append((word.lemma, sent.words[i + 1].lemma))
                if (word.pos, ) in patterns:
                    res.append((word.lemma, ))
    return res

In [38]:
candidates = get_mapped_patterns(data, patterns)

In [47]:
def clean_lst(lst, cands):
    no = [[tuple(el1[0].split(' ')) for el1 in el] for el in lst]
    yes = [[el1 for el1 in el if el1 in cands] for el in no]
    return no, yes

In [48]:
rake_raw_no, rake_raw_pt = clean_lst(rake_lst_raw, candidates)
rake_cl_no, rake_cl_pt = clean_lst(rake_lst_cl, candidates)
trank_raw_no, trank_raw_pt = clean_lst(trank_lst_raw, candidates)
trank_cl_no, tarank_cl_pt = clean_lst(trank_lst_cl, candidates)
kbert_raw_no, kbert_raw_pt = clean_lst(kbert_lst_raw, candidates)
kbert_cl_no, kbert_cl_pt = clean_lst(kbert_lst_cl, candidates)

# Eval
Оцениваем результаты

In [105]:
def get_kws_from_data(data):
    kws = []
    for text in data:
        kws.append([])
        for sent in text.sentences:
            kw = []
            for word in sent.words:
                if word.BIO == 'I':
                    kw[-1].append(word.lemma)
                elif word.BIO == 'B':
                    kw.append([word.lemma, ])
            for k in kw:
                if tuple(k) not in kws[-1]:
                    kws[-1].append(tuple(k))
    return kws

In [108]:
gold = get_kws_from_data(data)

In [132]:
def get_accuracy(golds, cond):
    acc = []
    for i, gold in enumerate(golds):
        if len(set(cond[i])) > 0:
            acc.append(len(set(gold) & set(cond[i])) / len(set(cond[i])))
        else:
            acc.append(0)
    return sum(acc) / len(acc)

In [133]:
def get_recall(golds, cond):
    rcc = []
    for i, gold in enumerate(golds):
        rcc.append(len(set(gold) & set(cond[i])) / len(set(gold)))
    return sum(rcc) / len(rcc)

In [134]:
def get_fscore(acc, rcc):
    if acc + rcc != 0:
        fsc = 2 * acc * rcc / (acc + rcc)
    else:
        fsc = 0
    return fsc

In [135]:
def get_for_all(lst, gold):
    res = []
    for el in lst:
        acc = get_accuracy(gold, el)
        rcc = get_recall(gold, el)
        fsc = get_fscore(acc, rcc)
        res.append([acc, rcc, fsc])
    return res

In [145]:
names = ['rake_raw_no', 'rake_raw_pt', 'rake_cl_no', 'rake_cl_pt', 'trank_raw_no', 
         'trank_raw_pt', 'trank_cl_no', 'trank_cl_pt', 'kbert_raw_no', 'kbert_raw_pt', 
         'kbert_cl_no', 'kbert_cl_pt']
lst = [rake_raw_no, rake_raw_pt, rake_cl_no, rake_cl_pt, trank_raw_no, trank_raw_pt, 
       trank_cl_no, tarank_cl_pt, kbert_raw_no, kbert_raw_pt, kbert_cl_no, kbert_cl_pt]
df = pd.DataFrame(get_for_all(lst, gold), columns=['precision', 'recall', 'f-score'], 
                 index=names)
df.sort_values(by='f-score', ascending=False)

,precision,recall,f-score
trank_raw_pt,0.349801,0.256132,0.295726
rake_cl_pt,0.527778,0.098399,0.165873
trank_cl_pt,0.133927,0.189261,0.156857
rake_cl_no,0.269608,0.098399,0.144177
trank_cl_no,0.078792,0.202082,0.113377
trank_raw_no,0.059710,0.276528,0.098213
kbert_cl_pt,0.055556,0.020833,0.030303
kbert_cl_no,0.033333,0.020833,0.025641
rake_raw_no,0.000000,0.000000,0.000000
rake_raw_pt,0.000000,0.000000,0.000000


Вообще, интересная получилась история: ожидаемо лучшее качество у тех, где мы заранее отобрали пошаблону. Ну, это и логично, так как я знала, что имела в виду при разметке, когда составляла шаблоны. А вот то, что самым первым оказался trank без очистки от стоп слов и всякой прочей бяки, вот это удивительно. Хотя он в целом оказался лучшим если смотреть по сумме позиций. А вот kbert разочаровал: он мало того, что долгий, так еще и качество дал никакое.

# Analysis

In [148]:
def get_lost(gold, cond):
    lst = []
    for i, gold in enumerate(gold):
        lst.append(set(gold) - set(cond[i]))
    return lst

In [154]:
def get_all_lost(lst, gold):
    res = []
    for el in lst:
        lst = get_lost(gold, el)
        res.append(list(lst))
    return res

In [158]:
 losst = get_all_lost(lst, gold)
 for i, name in enumerate(names):
     print('*' * 100)
     print(name)
     pprint(losst[i])

****************************************************************************************************
rake_raw_no
[{('гаджет',),
  ('компонент',),
  ('ноутбук',),
  ('отход',),
  ('перерабатывать',),
  ('переработка',),
  ('переработка', 'электронный', 'мусор'),
  ('повторный', 'использование'),
  ('ремонт', 'электроника'),
  ('смартфон',),
  ('телевизор',),
  ('устройство',),
  ('утилизация',),
  ('утилизация', 'старый', 'устройство'),
  ('утилизация', 'электронный', 'мусор'),
  ('утилизировать', 'техника'),
  ('утилизировать', 'устройство'),
  ('электронный', 'компонент'),
  ('электронный', 'мусор'),
  ('электронный', 'отход'),
  ('электронный', 'устройство'),
  ('электротехнический', 'отход')},
 {('встроить', 'проектор'),
  ('карманный', 'проектор'),
  ('карманный', 'размер'),
  ('карманный', 'формат'),
  ('миниатюрный', 'проекционный', 'система'),
  ('пико-проектор',),
  ('пикопроектор',),
  ('портативный', 'проектор'),
  ('проектор',),
  ('проецирование',),
  ('проецирование', 'изо

Какие есть проблемы:
1. Есть ощущение, что я выделдила больше ключевых слов, чем модель в целом хочет находить. Я скорее выделяла паттерны для поиска или индексации, чем для определения тематики, из-за этого получилось более подробно и модели с этим не справились
2. Модели, особенно на неочищенных данных, очень много выделяли мусора и из-за этого теряли нужное. Так что данные надо чистить.
3. Возможно, стоит определять и удалять специфичексие для тематики стоп-слова типа устройство, хотя я выделяла их как ключевые, так как при смешанных текстах это важно. И при работе с биграммами+